# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filename = "output_data/cities.csv"
df = pd.read_csv(filename)
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san quintin,30.4833,-115.9500,75.06,29,74,7.72,MX,1644699245
1,1,ushuaia,-54.8000,-68.3000,47.86,53,40,42.57,AR,1644699205
2,2,rikitea,-23.1203,-134.9692,79.12,70,2,15.86,PF,1644699245
3,3,port hawkesbury,45.6169,-61.3485,49.95,87,75,12.66,CA,1644699246
4,4,new norfolk,-42.7826,147.0587,59.76,67,100,3.71,AU,1644699246
...,...,...,...,...,...,...,...,...,...,...
520,520,vestmanna,62.1564,-7.1664,36.52,93,75,0.00,FO,1644700052
521,521,darnah,32.7670,22.6367,55.65,58,0,7.34,LY,1644700052
522,522,padang,-0.9492,100.3543,76.21,81,96,4.16,ID,1644700053
523,523,caraquet,47.7941,-64.9386,35.83,100,100,9.53,CA,1644700053


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = df[['Lat','Lng']]
weights = df['Humidity'].astype(float)
intensity = df['Humidity'].max()
fig = gmaps.figure(center = [0,0], zoom_level = 2)

heatmap = gmaps.heatmap_layer(locations, weights, max_intensity = intensity, point_radius = 1.5, dissipating = False)

fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = df[(df["Wind Speed"] >= 5) & (df["Wind Speed"] <= 15) & (df["Cloudiness"] >= 30) & (df["Cloudiness"] <= 45) & 
   (df["Max Temp"] >= 65) & (df["Max Temp"] <= 80)] 
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,kapaa,22.0752,-159.3190,77.11,70,40,9.22,US,1644699070
86,86,vila velha,-20.3297,-40.2925,76.95,92,40,5.75,BR,1644699328
273,273,saint-philippe,-21.3585,55.7679,78.35,82,31,11.21,RE,1644699684
439,439,antalaha,-14.9003,50.2788,73.15,92,33,5.41,MG,1644699910


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather.reset_index(drop = True)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,18,kapaa,22.0752,-159.3190,77.11,70,40,9.22,US,1644699070,
1,86,vila velha,-20.3297,-40.2925,76.95,92,40,5.75,BR,1644699328,
2,273,saint-philippe,-21.3585,55.7679,78.35,82,31,11.21,RE,1644699684,
3,439,antalaha,-14.9003,50.2788,73.15,92,33,5.41,MG,1644699910,


In [7]:
parameters = {"radius": 5000,
              "types": "hotel",
              "keyword": "hotels",
              "key": g_key
    
}

In [8]:
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    parameters["location"] = f"{latitude},{longitude}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(url,params = parameters)
    hotelname = hotels.json()
    hotel_df.loc[index, 'Hotel Name'] = hotelname["results"][0]["name"]
    
#    print(hotelname)

In [9]:
#Cleaned
hotel_df = hotel_df.drop('Unnamed: 0',axis = 1)

In [10]:
hotel_df
hotelfilename = "output_data/hotels.csv"
hotel_df.to_csv(hotelfilename,index = False) 

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Kauai Shores Hotel</dd>\n<dt>City</dt><dd>kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Golden Tulip Porto Vitória</dd>\n<dt>City</dt><dd>vila velha</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Pinpin d'Amour</dd>\n<dt>City</dt><dd>saint-philippe</dd>\n<dt>Country</dt><dd>RE</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Belle rose beach Hotel</dd>\n<dt>City</dt><dd>antalaha</dd>\n<dt>Country</dt><dd>MG</dd>\n</dl>\n']

In [13]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))